## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,39.27,90,0,1.99,clear sky
1,1,Bluff,NZ,-46.6000,168.3333,40.80,79,98,7.83,overcast clouds
2,2,Kavieng,PG,-2.5744,150.7967,80.94,79,100,9.95,light rain
3,3,Boone,US,39.0334,-92.3335,90.79,36,0,5.75,clear sky
4,4,Hovd,MN,48.0056,91.6419,64.92,43,83,2.86,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
#6.5.3 ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
#6.5.3 filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Kavieng,PG,-2.5744,150.7967,80.94,79,100,9.95,light rain
5,5,Mangrol,IN,21.1167,70.1167,81.88,81,100,16.17,overcast clouds
6,6,Yining,CN,29.0361,114.5636,79.79,88,98,2.53,overcast clouds
14,14,Dwarka,IN,22.2394,68.9678,83.08,81,98,18.52,overcast clouds
18,18,Bukan,IR,36.5210,46.2089,88.45,12,0,19.75,clear sky
28,28,Morehead,US,37.2711,-87.1764,85.98,64,0,3.44,clear sky
32,32,Ugoofaaru,MV,5.6667,73.0000,82.18,77,100,17.81,overcast clouds
33,33,Pangai,TO,-19.8000,-174.3500,76.69,69,0,11.92,clear sky
34,34,Faanui,PF,-16.4833,-151.7500,76.71,75,17,12.80,few clouds
36,36,Lorengau,PG,-2.0226,147.2712,80.17,78,99,4.09,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
#6.5.3 determine if there are any null values
preferred_cities_df.count()

City_ID                249
City                   249
Country                248
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.count()

City_ID                248
City                   248
Country                248
Lat                    248
Lng                    248
Max Temp               248
Humidity               248
Cloudiness             248
Wind Speed             248
Current Description    248
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kavieng,PG,80.94,light rain,-2.5744,150.7967,
5,Mangrol,IN,81.88,overcast clouds,21.1167,70.1167,
6,Yining,CN,79.79,overcast clouds,29.0361,114.5636,
14,Dwarka,IN,83.08,overcast clouds,22.2394,68.9678,
18,Bukan,IR,88.45,clear sky,36.5210,46.2089,
28,Morehead,US,85.98,clear sky,37.2711,-87.1764,
32,Ugoofaaru,MV,82.18,overcast clouds,5.6667,73.0000,
33,Pangai,TO,76.69,clear sky,-19.8000,-174.3500,
34,Faanui,PF,76.71,few clouds,-16.4833,-151.7500,
36,Lorengau,PG,80.17,overcast clouds,-2.0226,147.2712,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kavieng,PG,80.94,light rain,-2.5744,150.7967,Nusa Island Retreat
5,Mangrol,IN,81.88,overcast clouds,21.1167,70.1167,Hotel The Royal Honours
6,Yining,CN,79.79,overcast clouds,29.0361,114.5636,Xiushui Hotel
14,Dwarka,IN,83.08,overcast clouds,22.2394,68.9678,Hotel Narayan Avezika Comfort
18,Bukan,IR,88.45,clear sky,36.5210,46.2089,فروشگاه پارچه صدیقی
28,Morehead,US,85.98,clear sky,37.2711,-87.1764,CCI Express Inn
32,Ugoofaaru,MV,82.18,overcast clouds,5.6667,73.0000,Ungoofaaru Inn
33,Pangai,TO,76.69,clear sky,-19.8000,-174.3500,Ha'apai Beach Resort
34,Faanui,PF,76.71,few clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
36,Lorengau,PG,80.17,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd><{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd><{Country}</dd>
<dt>Weather Description</dt><dd{Current Description}</dd>
<dt>Max Temp</dt><dd><{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))